In [ ]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger

In [ ]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 2

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In [ ]:
# Customize the logger format
logger.remove()
logger.add(
    sink=lambda msg: print(msg, end=""),
    format="<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
           "<level>{level: <8}</level> | "
           "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
           "{message}",
    colorize=True,  # Enable colors for supported terminals
)

In [ ]:
#Manually resets the index on execution
# recreate_index() 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [ ]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [ ]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

3 - Queries Iroha 1 for User account and checks its values

In [ ]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail

logger.info(f'User Account id = {user_account}, {user_details}')

6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [ ]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

In [ ]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

In [ ]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


In [ ]:
# Perform a keyword search
keyword = "paper"
search_results, project_ids_with_cids = search_index(index, keyword)

In [ ]:
# Check if there are no search results
if not search_results:
    logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
else:
    # Process each dictionary in search results
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        if not project_id or not file_cid or not metadata_cid:
            logger.error(f"Missing required data in result: {result_dict}")
            continue

        # Log the retrieved project details
        logger.info(f"Processing Project ID: {project_id}")
        logger.info(f"File CID: {file_cid}")
        logger.info(f"Metadata CID: {metadata_cid}")
        # file_metadata_json = download_json_from_ipfs(metadata_cid)
        # logger.info("file_metadata_json:", file_metadata_json)
        

        # Fetch project details from the blockchain
        project_details = get_account_detail(project_id)
        if not project_details:
            logger.error(f"No project details found for Project ID: {project_id}.")
            continue

        logger.info(f"Fetched project details for {project_id}: {project_details}")

        # Parse blockchain data
        try:
            blockchain_data = json.loads(project_details)
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding project details JSON for {project_id}: {e}")
            continue

        # Validate file CID and fetch project details
        validation_result = fetch_project_details(file_cid, blockchain_data)
        logger.info(f"Valid Result for {project_id} is {validation_result}.")
        if validation_result["is_valid"]:
            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")
            file_metadata_cid = validation_result.get("metadata_cid")
            
            # download_file(file_metadata_json, download_path, project_id, file_cid)

            logger.info(f"Valid File CID for {project_id}.")
            logger.info(f"Project Metadata CID: {project_metadata_cid}")
            logger.info(f"Linked User: {linked_user}")

            # Fetch and process metadata and user details
            if project_metadata_cid:
                logger.info(f"Processing project metadata CID: {project_metadata_cid}")
                project_metadata = download_json_from_ipfs(project_metadata_cid)
                logger.info(f"Downloaded project metadata: {project_metadata}")

            if linked_user:
                logger.info(f"Processing linked user: {linked_user}")
                user_details = get_account_detail(linked_user)
                try:
                    user_details = json.loads(user_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                    continue

                user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid", None)
                if user_json_ld_cid:
                    logger.info(f"User JSON-LD CID: {user_json_ld_cid}")
                    user_metadata = download_json_from_ipfs(user_json_ld_cid)
                    logger.info(f"Downloaded user metadata: {user_metadata}")
                else:
                    logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")
            
            if metadata_cid:
                logger.info(f"Processing metadata CID: {metadata_cid}")
                file_metadata = download_json_from_ipfs(metadata_cid)
                file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                logger.info(f"Downloaded file metadata: {metadata_cid}")
                logger.info(f"file metadata: {file_metadata}")
                logger.info(f"file metadata JSON: {file_metadata_json}")
            
        
        else:
            logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")

With Reusable Helper Function for Block Separation and Tracing

A helper function, with_logging_block, which accepts a block name, the code block to execute, and optional parameters for context-specific tracing.

Cleanly separates each logical block, improving readability, modularity, and error tracing.

In [ ]:
from contextlib import contextmanager
import json
import sys


# Main Code Logic

with with_logging_block("Keyword Search", logger):
    if not search_results:
        logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
        sys.exit(1)

with with_logging_block("Processing Search Results", logger):
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        with with_logging_block(f"Processing Result for Project ID: {project_id or 'Unknown'}", logger):
            if not project_id or not file_cid or not metadata_cid:
                logger.error(f"Missing required data in result: {result_dict}")
                continue

            logger.info(f"File CID: {file_cid}")
            logger.info(f"Metadata CID: {metadata_cid}")

            # Fetch project details
            with with_logging_block("Fetching Project Details", logger):
                project_details = get_account_detail(project_id)
                if not project_details:
                    logger.error(f"No project details found for Project ID: {project_id}.")
                    continue
                logger.info(f"Fetched project details for {project_id}: {project_details}")

            # Parse blockchain data
            with with_logging_block("Parsing Blockchain Data", logger):
                try:
                    blockchain_data = json.loads(project_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding project details JSON for {project_id}: {e}")
                    continue

            # Validate file CID
            with with_logging_block("Validating File CID", logger):
                validation_result = fetch_project_details(file_cid, blockchain_data)
                logger.info(f"Validation Result for {project_id}: {validation_result}")
                if not validation_result["is_valid"]:
                    logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")
                    continue

            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")

            # Process project metadata
            if project_metadata_cid:
                with with_logging_block("Processing Project Metadata", logger):
                    project_metadata = download_json_from_ipfs(project_metadata_cid)
                    logger.info(f"Downloaded project metadata: {project_metadata}")

            # Process linked user details
            if linked_user:
                with with_logging_block(f"Processing Linked User: {linked_user}", logger):
                    user_details = get_account_detail(linked_user)
                    try:
                        user_details = json.loads(user_details)
                    except json.JSONDecodeError as e:
                        logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                        continue

                    user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid")
                    if user_json_ld_cid:
                        user_metadata = download_json_from_ipfs(user_json_ld_cid)
                        logger.info(f"Downloaded user metadata: {user_metadata}")
                    else:
                        logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")

            # Process metadata CID
            if metadata_cid:
                with with_logging_block("Processing Metadata CID", logger):
                    file_metadata = download_json_from_ipfs(metadata_cid)
                    file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
             


includes the output of the search results explicitly logged within the Keyword Search block. This will ensure all search results are printed for visibility and debugging purposes.

In [32]:
from contextlib import contextmanager
import json
import sys


@contextmanager
def with_logging_block(block_name, logger):
    """
    A reusable context manager for logging structured execution blocks.

    Args:
        block_name (str): The name of the block being executed.
        logger (Logger): The logger instance.
    """
    try:
        logger.info("\n" + "=" * 50)
        logger.info(f"STARTING BLOCK: {block_name}")
        logger.info("=" * 50)
        yield  # Code within the `with` block will execute here
    except Exception as e:
        logger.error(f"An error occurred in block '{block_name}': {e}. Exiting.")
        sys.exit(1)  # Graceful exit on error
    finally:
        logger.info(f"COMPLETED BLOCK: {block_name}")
        logger.info("-" * 50 + "\n")


# Main Code Logic

with with_logging_block("Keyword Search", logger):
    if not search_results:
        logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
        sys.exit(1)
    logger.info(f"Search results for keyword '{keyword}':")
    for idx, result in enumerate(search_results, 1):
        logger.info(f"Result {idx}: {json.dumps(result, indent=2)}")

with with_logging_block("Processing Search Results", logger):
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        with with_logging_block(f"Processing Result for Project ID: {project_id or 'Unknown'}", logger):
            if not project_id or not file_cid or not metadata_cid:
                logger.error(f"Missing required data in result: {result_dict}")
                continue

            logger.info(f"File CID: {file_cid}")
            logger.info(f"Metadata CID: {metadata_cid}")

            # Fetch project details
            with with_logging_block("Fetching Project Details", logger):
                project_details = get_account_detail(project_id)
                if not project_details:
                    logger.error(f"No project details found for Project ID: {project_id}.")
                    continue
                logger.info(f"Fetched project details for {project_id}: {project_details}")

            # Parse blockchain data
            with with_logging_block("Parsing Blockchain Data", logger):
                try:
                    blockchain_data = json.loads(project_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding project details JSON for {project_id}: {e}")
                    continue

            # Validate file CID
            with with_logging_block("Validating File CID", logger):
                validation_result = fetch_project_details(file_cid, blockchain_data)
                logger.info(f"Validation Result for {project_id}: {validation_result}")
                if not validation_result["is_valid"]:
                    logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")
                    continue

            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")

            # Process project metadata
            if project_metadata_cid:
                with with_logging_block("Processing Project Metadata", logger):
                    project_metadata = download_json_from_ipfs(project_metadata_cid)
                    logger.info(f"Downloaded project metadata: {project_metadata}")

            # Process linked user details
            if linked_user:
                with with_logging_block(f"Processing Linked User: {linked_user}", logger):
                    user_details = get_account_detail(linked_user)
                    try:
                        user_details = json.loads(user_details)
                    except json.JSONDecodeError as e:
                        logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                        continue

                    user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid")
                    if user_json_ld_cid:
                        user_metadata = download_json_from_ipfs(user_json_ld_cid)
                        logger.info(f"Downloaded user metadata: {user_metadata}")
                    else:
                        logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")

            # Process metadata CID
            if metadata_cid:
                with with_logging_block("Processing Metadata CID", logger):
                    file_metadata = download_json_from_ipfs(metadata_cid)
                    file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                    logger.info(f"Downloaded file metadata: {file_metadata_json}")


2025-01-13 22:51:07.473 | INFO     | __main__:with_logging_block:16 - 
2025-01-13 22:51:07.473 | INFO     | __main__:with_logging_block:17 - STARTING BLOCK: Keyword Search
2025-01-13 22:51:07.473 | INFO     | __main__:with_logging_block:18 - ==================================================
2025-01-13 22:51:07.474 | INFO     | __main__:<module>:34 - Search results for keyword 'paper':
2025-01-13 22:51:07.475 | INFO     | __main__:<module>:36 - Result 1: {
  "abstract": "",
  "created": "2023-02-04t09:40:15z",
  "creator": "chenxun yuan, xiang ma, hua wang, caiming zhang, xuemei li",
  "date": "",
  "description": "expert systems with applications, 217 (2023) 119549. doi:10.1016/j.eswa.2023.119549",
  "file_cid": "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW",
  "format": "application/pdf; version=1.7",
  "full_text": "\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCOVID19-MLSF: A multi-task learning